In [4]:
! pip install -r requirements.txt

In [5]:
import pandas as pd
import geopandas as gpd

In [6]:
HI_cal = pd.read_csv('./Hawaii/calendar.csv.gz', compression='gzip',)
HI_lis = pd.read_csv('./Hawaii/airbnb/hawaii_listings.csv')
HI_nei = pd.read_csv('./Hawaii/neighbourhoods.csv').fillna(0)

display(HI_cal.head(5))
display(HI_lis.head(5))
display(HI_nei.head(5))

print(HI_lis.shape)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5065,2021-07-09,f,$85.00,$85.00,2.0,700.0
1,87671,2021-07-09,f,$227.00,$227.00,7.0,1125.0
2,87671,2021-07-10,f,$227.00,$227.00,7.0,1125.0
3,87671,2021-07-11,f,$227.00,$227.00,7.0,1125.0
4,87671,2021-07-12,f,$227.00,$227.00,7.0,1125.0


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions
0,5065,https://www.airbnb.com/rooms/5065,2.020000e+13,7/9/2021,MAUKA BB,"Perfect for your vacation, Staycation or just ...",Neighbors here are friendly but are not really...,https://a0.muscache.com/pictures/36718112/1f0e...,7257,https://www.airbnb.com/users/show/7257,...,4.48,4.76,NaN,f,1,1,0,0,0.41,30
1,5269,https://www.airbnb.com/rooms/5269,2.020000e+13,7/9/2021,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","We are located on the ""sunny side"" of Waimea, ...",https://a0.muscache.com/pictures/5b52b72f-5a09...,7620,https://www.airbnb.com/users/show/7620,...,5.00,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36
2,5387,https://www.airbnb.com/rooms/5387,2.020000e+13,7/9/2021,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,IN a Farm belt area with small commercial farm...,https://a0.muscache.com/pictures/1170713/dca6a...,7878,https://www.airbnb.com/users/show/7878,...,4.72,4.74,NaN,t,3,3,0,0,1.66,24
3,5389,https://www.airbnb.com/rooms/5389,2.020000e+13,7/9/2021,Keauhou Villa,It is less than 10 minute walk to the Keauhou ...,It is less than 10 minute walk to the Keauhou ...,https://a0.muscache.com/pictures/15520396/3b89...,7878,https://www.airbnb.com/users/show/7878,...,4.97,4.74,NaN,f,3,3,0,0,0.57,23
4,5390,https://www.airbnb.com/rooms/5390,2.020000e+13,7/9/2021,STAY AT PRINCE KUHIO!,"<b>The space</b><br />Prince Kuhio, Studio Uni...",NaN,https://a0.muscache.com/pictures/12955/af97ac5...,7887,https://www.airbnb.com/users/show/7887,...,4.90,4.67,42652226,f,1,1,0,0,2.07,92


,neighbourhood_group,neighbourhood
0,Hawaii,Hamakua
1,Hawaii,Kau
2,Hawaii,North Hilo
3,Hawaii,North Kohala
4,Hawaii,North Kona


(21808, 75)


In [5]:
def my_szn(date):
    mon = date.month
    day = date.day
    
    if mon < 3 or mon == 3 and day <=20:
        szn='Winter'
    elif mon < 6 or mon == 6 and day <=20:
        szn = 'Spring'
    elif mon < 9 or mon == 9 and day <=20:
        szn = 'Summer'
    elif mon < 12 or mon == 12 and day <=20:
        szn = 'Fall'
    else:
        szn='Winter'
    
    return szn


In [6]:
HI_cal['date'] = pd.to_datetime(HI_cal['date'], infer_datetime_format=True)
HI_cal['month'] = HI_cal['date'].dt.month
HI_cal['qtr'] = HI_cal['date'].dt.quarter
HI_cal['is_weekend'] = HI_cal['date'].dt.weekday >= 4
HI_cal['dayofweek'] = HI_cal['date'].dt.dayofweek




HI_cal['szn'] = HI_cal.apply(lambda x: my_szn(x['date']), axis=1 )



In [7]:
HI_cal_piv = HI_cal.groupby(['listing_id','szn','is_weekend','available'])\
            .agg({'price':'count'}).reset_index()\
            .pivot(['listing_id'],['szn','is_weekend','available'],['price'])\
            .reset_index().fillna(0)

HI_cal_piv.columns = HI_cal_piv.columns.to_flat_index()
HI_cal_piv.columns = ['listing_id',
 'Fall_weekday_f', 'Fall_weekday_t', 'Fall_weekend_f', 'Fall_weekend_t',
 'Spring_weekday_f', 'Spring_weekday_t', 'Spring_weekend_f', 'Spring_weekend_t',
 'Summer_weekday_f', 'Summer_weekday_t', 'Summer_weekend_f', 'Summer_weekend_t',
 'Winter_weekday_f', 'Winter_weekday_t', 'Winter_weekend_f', 'Winter_weekend_t']

HI_cal_piv['Fall_weekday'] = HI_cal_piv['Fall_weekday_t']/(HI_cal_piv['Fall_weekday_t']+HI_cal_piv['Fall_weekday_f'])
HI_cal_piv['Fall_weekend'] = HI_cal_piv['Fall_weekend_t']/(HI_cal_piv['Fall_weekend_t']+HI_cal_piv['Fall_weekend_f'])

HI_cal_piv['Spring_weekday'] = HI_cal_piv['Spring_weekday_t']/(HI_cal_piv['Spring_weekday_t']+HI_cal_piv['Spring_weekday_f'])
HI_cal_piv['Spring_weekend'] = HI_cal_piv['Spring_weekend_t']/(HI_cal_piv['Spring_weekend_t']+HI_cal_piv['Spring_weekend_f'])

HI_cal_piv['Summer_weekday'] = HI_cal_piv['Summer_weekday_t']/(HI_cal_piv['Summer_weekday_t']+HI_cal_piv['Summer_weekday_f'])
HI_cal_piv['Summer_weekend'] = HI_cal_piv['Summer_weekend_t']/(HI_cal_piv['Summer_weekend_t']+HI_cal_piv['Summer_weekend_f'])

HI_cal_piv['Winter_weekday'] = HI_cal_piv['Winter_weekday_t']/(HI_cal_piv['Winter_weekday_t']+HI_cal_piv['Winter_weekday_f'])
HI_cal_piv['Winter_weekend'] = HI_cal_piv['Winter_weekend_t']/(HI_cal_piv['Winter_weekend_t']+HI_cal_piv['Winter_weekend_f'])

HI_cal_piv['Fall'] = HI_cal_piv[[ 'Fall_weekday_t', 'Fall_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Fall_weekday_f', 'Fall_weekday_t', 'Fall_weekend_f', 'Fall_weekend_t',]].sum(axis=1))
HI_cal_piv['Spring'] = HI_cal_piv[[ 'Spring_weekday_t', 'Spring_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Spring_weekday_f', 'Spring_weekday_t', 'Spring_weekend_f', 'Spring_weekend_t',]].sum(axis=1))
HI_cal_piv['Summer'] = HI_cal_piv[[ 'Summer_weekday_t', 'Summer_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Summer_weekday_f', 'Summer_weekday_t', 'Summer_weekend_f', 'Summer_weekend_t',]].sum(axis=1))
HI_cal_piv['Winter'] = HI_cal_piv[[ 'Winter_weekday_t', 'Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Winter_weekday_f', 'Winter_weekday_t', 'Winter_weekend_f', 'Winter_weekend_t']].sum(axis=1))


HI_cal_piv['Weekday'] = HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekday_f','Spring_weekday_f','Summer_weekday_f','Winter_weekday_f',]].sum(axis=1))

HI_cal_piv['Weekend'] = HI_cal_piv[['Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekend_f','Spring_weekend_f','Summer_weekend_f','Winter_weekend_f',]].sum(axis=1))

HI_cal_piv['NoVacancy']=HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekday_f','Spring_weekday_f','Summer_weekday_f','Winter_weekday_f',
                                    'Fall_weekend_f','Spring_weekend_f','Summer_weekend_f','Winter_weekend_f',]].sum(axis=1))

HI_cal_piv['Occupancy']=HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)



HI_cal_piv = HI_cal_piv[['listing_id','Occupancy','Fall_weekday','Fall_weekend','Spring_weekday','Spring_weekend',
                         'Summer_weekday','Summer_weekend','Winter_weekday','Winter_weekend',
                         'Fall','Spring','Summer','Winter',
                         'Weekday','Weekend','NoVacancy']]

HI_cal_piv

,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,Summer_weekday,Summer_weekend,Winter_weekday,Winter_weekend,Fall,Spring,Summer,Winter,Weekday,Weekend,NoVacancy
0,5065,146.0,0.557692,0.538462,0.423913,0.211538,0.175,0.433333,NaN,NaN,0.549451,0.347222,0.353846,NaN,0.407609,0.392265,0.400000
1,5269,154.0,0.211538,0.205128,0.423913,0.615385,0.625,0.450000,0.428571,NaN,0.208791,0.493056,0.566667,0.428571,0.413386,0.441441,0.421918
2,5387,135.0,0.307692,0.282051,0.423913,0.346154,0.300,0.333333,0.476190,NaN,0.296703,0.395833,0.313433,0.476190,0.392713,0.322034,0.369863
3,5389,127.0,0.230769,0.230769,0.423913,0.384615,0.200,0.333333,0.448718,NaN,0.230769,0.409722,0.230769,0.448718,0.358779,0.320388,0.347945
4,5390,102.0,0.096154,0.076923,0.402439,0.173077,0.175,0.440000,0.430769,0.600000,0.087912,0.313433,0.276923,0.453333,0.305439,0.230159,0.279452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21803,50898372,157.0,0.403846,0.410256,0.423529,0.500000,0.400,0.453125,0.384615,0.428571,0.406593,0.452555,0.432692,0.393939,0.408867,0.456790,0.430137
21804,50898551,155.0,0.461538,0.384615,0.423913,0.480769,0.325,0.428571,0.439024,NaN,0.428571,0.444444,0.382022,0.439024,0.417778,0.435714,0.424658
21805,50900058,187.0,0.673077,0.794872,0.423913,0.442308,0.500,0.440000,0.430769,NaN,0.725275,0.430556,0.476923,0.430769,0.489960,0.560345,0.512329
21806,50900602,102.0,0.115385,0.153846,0.405405,0.153846,0.100,1.000000,0.413793,0.436364,0.131868,0.258427,0.162791,0.422535,0.282407,0.275168,0.279452


In [28]:
HI_df = HI_cal_piv.merge(HI_lis, left_on='listing_id', right_on='id', how='inner', suffixes=('','_lis'))
HI_df = HI_df.merge(HI_nei.drop('neighbourhood',axis=1), on=['neighbourhood_group'], how='inner', suffixes=('','_nei')).drop_duplicates()
print(HI_df.columns, HI_df.shape)
HI_df.head(5)


Index(['listing_id', 'Occupancy', 'Fall_weekday', 'Fall_weekend',
       'Spring_weekday', 'Spring_weekend', 'Summer_weekday', 'Summer_weekend',
       'Winter_weekday', 'Winter_weekend', 'Fall', 'Spring', 'Summer',
       'Winter', 'Weekday', 'Weekend', 'NoVacancy', 'id', 'listing_url',
       'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'bed

,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,Summer_weekday,Summer_weekend,Winter_weekday,Winter_weekend,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions
0,5065,146.0,0.557692,0.538462,0.423913,0.211538,0.175,0.433333,NaN,NaN,...,4.48,4.76,NaN,f,1,1,0,0,0.41,30
9,5269,154.0,0.211538,0.205128,0.423913,0.615385,0.625,0.450000,0.428571,NaN,...,5.00,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36
18,5387,135.0,0.307692,0.282051,0.423913,0.346154,0.300,0.333333,0.476190,NaN,...,4.72,4.74,NaN,t,3,3,0,0,1.66,24
27,5389,127.0,0.230769,0.230769,0.423913,0.384615,0.200,0.333333,0.448718,NaN,...,4.97,4.74,NaN,f,3,3,0,0,0.57,23
36,8833,120.0,0.192308,0.102564,0.437500,0.307692,0.300,0.285714,0.445783,0.333333,...,4.74,4.58,W95781870,f,1,1,0,0,2.54,23


In [29]:
HI_poi_rest = pd.read_csv("./Hawaii/AK_Air_Rewards_Restraunts_HI.csv",header=None)[[0,1,2]]
HI_poi_mili = pd.read_csv("./Hawaii/Hawaii historic military sites.csv",header=None)[[0,1,2]]
HI_poi_beaK = pd.read_csv("./Hawaii/Kauai Beaches.csv",header=None)[[0,1,2]]
HI_poi_sigK = pd.read_csv("./Hawaii/Kauai Sights.csv",header=None)[[0,1,2]]
HI_poi_shoK = pd.read_csv("./Hawaii/Kauai Shopping.csv",header=None)[[0,1,2]]
HI_poi_beaO = pd.read_csv("./Hawaii/Oahu Beaches.csv",header=None)[[0,1,2]]
HI_poi_sigO = pd.read_csv("./Hawaii/Oahu Sights.csv",header=None)[[0,1,2]]
HI_poi_shoO = pd.read_csv("./Hawaii/Oahu Shopping.csv",header=None)[[0,1,2]]
HI_poi_snor = pd.read_csv("./Hawaii/Snorkel Bob.csv",header=None)[[0,1,2]]
HI_poi_tour = pd.read_csv("./Hawaii/USAPOI Tourist Guide for _Hawaii.csv",header=None)[[0,1,2]]


HI_poi = pd.concat([HI_poi_rest, HI_poi_mili, HI_poi_beaK, HI_poi_sigK, HI_poi_shoK,
                    HI_poi_beaO, HI_poi_sigO, HI_poi_shoO, HI_poi_snor, HI_poi_tour ])

HI_poi.columns = ['POI_LNG','POI_LAT','POI_name']
HI_poi

,POI_LNG,POI_LAT,POI_name
0,-159.320679,22.061848,La Playita Azul
1,-157.820877,21.273705,Matteo s Italian
2,-157.850525,21.292034,Mocha Java Cafe
3,-156.677338,20.872435,Mulligan s At The Wharf
4,-156.437607,20.674744,Mulligan s On The
...,...,...,...
39,-155.881076,19.418295,South Kona Fruit Stand
40,-155.995168,19.643350,Kona Tacos
41,-159.670837,21.958202,Daniel Thiebaut
42,-155.070245,19.725163,Nihon Restaurant


In [30]:
HI_poi_g = gpd.GeoDataFrame(HI_poi, geometry=gpd.points_from_xy(HI_poi.POI_LNG, HI_poi.POI_LAT), crs="EPSG:4326")
HI_poi_g = HI_poi_g.to_crs("EPSG:3857")
HI_poi_g

,POI_LNG,POI_LAT,POI_name,geometry
0,-159.320679,22.061848,La Playita Azul,POINT (-17735496.859 2518952.447)
1,-157.820877,21.273705,Matteo s Italian,POINT (-17568539.664 2424545.069)
2,-157.850525,21.292034,Mocha Java Cafe,POINT (-17571840.064 2426734.783)
3,-156.677338,20.872435,Mulligan s At The Wharf,POINT (-17441241.485 2376674.308)
4,-156.437607,20.674744,Mulligan s On The,POINT (-17414554.752 2353137.273)
...,...,...,...,...
39,-155.881076,19.418295,South Kona Fruit Stand,POINT (-17352602.005 2204245.815)
40,-155.995168,19.643350,Kona Tacos,POINT (-17365302.668 2230828.355)
41,-159.670837,21.958202,Daniel Thiebaut,POINT (-17774476.269 2506507.625)
42,-155.070245,19.725163,Nihon Restaurant,POINT (-17262340.711 2240500.975)


In [31]:
HI_df_g = gpd.GeoDataFrame(HI_df, geometry=gpd.points_from_xy(HI_df.longitude, HI_df.latitude), crs="EPSG:4326")
HI_df_g = HI_df_g.to_crs("EPSG:3857")
HI_df_g.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [32]:
HI_df_g['geometry'] = HI_df_g['geometry'].buffer(1600)
HI_f = gpd.sjoin(HI_poi_g, HI_df_g, how='right', op='within').sort_values(by=['id','POI_name'])
HI_f


,index_left,POI_LNG,POI_LAT,POI_name,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions,geometry
0,NaN,NaN,NaN,NaN,5065,146.0,0.557692,0.538462,0.423913,0.211538,...,4.76,NaN,f,1,1,0,0,0.41,30,"POLYGON ((-17301076.771 2278085.275, -17301084..."
9,NaN,NaN,NaN,NaN,5269,154.0,0.211538,0.205128,0.423913,0.615385,...,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36,"POLYGON ((-17331067.355 2276277.116, -17331075..."
18,39.0,-155.881076,19.418295,South Kona Fruit Stand,5387,135.0,0.307692,0.282051,0.423913,0.346154,...,4.74,NaN,t,3,3,0,0,1.66,24,"POLYGON ((-17350959.035 2205723.063, -17350966..."
27,3.0,-155.958868,19.561911,Ali'i Mountain Course at Kona Country Club,5389,127.0,0.230769,0.230769,0.423913,0.384615,...,4.74,NaN,f,3,3,0,0,0.57,23,"POLYGON ((-17360009.310 2221692.647, -17360017..."
43092,3.0,-159.476190,21.881470,Beach House (Kolopa) Beach,5390,102.0,0.096154,0.076923,0.402439,0.173077,...,4.67,42652226,f,1,1,0,0,2.07,92,"POLYGON ((-17750904.362 2497305.026, -17750912..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129012,NaN,NaN,NaN,NaN,50898372,157.0,0.403846,0.410256,0.423529,0.500000,...,NaN,GE-140-773-5808-01,t,137,137,0,0,NaN,60,"POLYGON ((-17440600.914 2386295.596, -17440608..."
129021,NaN,NaN,NaN,NaN,50898551,155.0,0.461538,0.384615,0.423913,0.480769,...,NaN,NaN,t,137,137,0,0,NaN,60,"POLYGON ((-17440744.906 2386214.659, -17440752..."
129030,NaN,NaN,NaN,NaN,50900058,187.0,0.673077,0.794872,0.423913,0.442308,...,NaN,NaN,t,137,137,0,0,NaN,60,"POLYGON ((-17420640.651 2367532.134, -17420648..."
129039,NaN,NaN,NaN,NaN,50900602,102.0,0.115385,0.153846,0.405405,0.153846,...,NaN,175-718-1952-01,t,261,257,4,0,NaN,60,"POLYGON ((-17441117.693 2385134.809, -17441125..."


In [33]:
HI_f['POI_name'].nunique()

296

In [34]:
HI_f[HI_f['POI_name']>'']

,index_left,POI_LNG,POI_LAT,POI_name,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions,geometry
18,39.0,-155.881076,19.418295,South Kona Fruit Stand,5387,135.0,0.307692,0.282051,0.423913,0.346154,...,4.74,NaN,t,3,3,0,0,1.66,24,"POLYGON ((-17350959.035 2205723.063, -17350966..."
27,3.0,-155.958868,19.561911,Ali'i Mountain Course at Kona Country Club,5389,127.0,0.230769,0.230769,0.423913,0.384615,...,4.74,NaN,f,3,3,0,0,0.57,23,"POLYGON ((-17360009.310 2221692.647, -17360017..."
43092,3.0,-159.476190,21.881470,Beach House (Kolopa) Beach,5390,102.0,0.096154,0.076923,0.402439,0.173077,...,4.67,42652226,f,1,1,0,0,2.07,92,"POLYGON ((-17750904.362 2497305.026, -17750912..."
43092,20.0,-159.472060,21.879580,Kuhio Park Beach,5390,102.0,0.096154,0.076923,0.402439,0.173077,...,4.67,42652226,f,1,1,0,0,2.07,92,"POLYGON ((-17750904.362 2497305.026, -17750912..."
43098,2.0,-159.464040,22.222890,Anini Beach,5434,165.0,0.538462,0.666667,0.428571,0.346154,...,4.74,5.40E+11,f,2,2,0,0,1.60,92,"POLYGON ((-17750506.952 2538055.636, -17750514..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180776,19.0,-157.831803,21.278370,SpaHalekulani at Halekulani,50897997,246.0,1.000000,0.948718,0.423913,0.903846,...,NaN,"260030540000, TA-100-079-4624-01",t,5,0,5,0,NaN,194,"POLYGON ((-17568094.715 2425349.768, -17568102..."
180776,22.0,-157.832703,21.289738,Tudo De Bom,50897997,246.0,1.000000,0.948718,0.423913,0.903846,...,NaN,"260030540000, TA-100-079-4624-01",t,5,0,5,0,NaN,194,"POLYGON ((-17568094.715 2425349.768, -17568102..."
180776,66.0,-157.827930,21.276300,Waikiki Beach,50897997,246.0,1.000000,0.948718,0.423913,0.903846,...,NaN,"260030540000, TA-100-079-4624-01",t,5,0,5,0,NaN,194,"POLYGON ((-17568094.715 2425349.768, -17568102..."
180776,30.0,-157.830930,21.279140,Waikiki Beach Walk,50897997,246.0,1.000000,0.948718,0.423913,0.903846,...,NaN,"260030540000, TA-100-079-4624-01",t,5,0,5,0,NaN,194,"POLYGON ((-17568094.715 2425349.768, -17568102..."


In [35]:
import math
def caluclate_distance_haversine(store_a_longitude, store_a_latitude, store_b_longitude, store_b_latitude):

    # Calculate Latitude and Longitude Difference
    latitude_difference = (store_b_latitude - store_a_latitude) * math.pi / 180.0
    longitude_difference = (store_b_longitude - store_a_longitude) * math.pi / 180.0
  
    # Convert decimal degrees to radians 
    store_a_latitude = (store_a_latitude) * math.pi / 180.0
    store_b_latitude = (store_b_latitude) * math.pi / 180.0
  
    # Compute Haversine Formula
    a = (pow(math.sin(latitude_difference / 2), 2) + 
         pow(math.sin(longitude_difference / 2), 2) * 
             math.cos(store_a_latitude) * math.cos(store_b_latitude)); 
    rad = 6371.0
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

In [36]:
fol_tmp = HI_f[['POI_LAT','POI_LNG','POI_name','id','latitude','longitude']]
fol_tmp['hav_dist_km'] = fol_tmp.apply(lambda x: 
                                    caluclate_distance_haversine(float(x.POI_LAT), float(x.POI_LNG), float(x.latitude),float(x.longitude))
                                    , axis=1)
fol_tmp['hav_dist_mi'] = fol_tmp['hav_dist_km'] * 0.621371
fol_tmp = fol_tmp[fol_tmp['hav_dist_mi']<=1.1]
fol_tmp.sort_values('hav_dist_mi').dropna()

,POI_LAT,POI_LNG,POI_name,id,latitude,longitude,hav_dist_km,hav_dist_mi
121398,20.916115,-156.694626,Son z Maui at,48781723,20.916130,-156.69464,0.002184,0.001357
164384,21.285540,-157.838898,Harbor Pub,43656485,21.285530,-157.83888,0.002251,0.001399
137072,21.274715,-157.824066,Popeye s Chicken,14934751,21.274740,-157.82407,0.002612,0.001623
178704,21.279428,-157.826385,Galaxy Steak,50123185,21.279427,-157.82634,0.004972,0.003089
46800,21.878580,-159.459390,Poipu Shopping Village,16596900,21.878630,-159.45940,0.005324,0.003308
...,...,...,...,...,...,...,...,...
138368,21.271389,-157.806111,Battery Harlow (historical),16571860,21.272860,-157.82037,1.592736,0.989680
135712,21.263450,-157.805060,Diamond Head State Monument,13282311,21.261970,-157.81932,1.592945,0.989810
170864,21.279573,-157.826569,Da Big Kahuna,47122899,21.279070,-157.81223,1.595265,0.991251
155344,21.289640,-157.848450,Ala Moana Park,35289736,21.289150,-157.83411,1.595334,0.991294


In [37]:
import folium
mapit=None
mapit = folium.Map(location=[21.9702,-159.36694], zoom_start=13)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(mapit)

small = 250 
fol_tmp = HI_f[HI_f['POI_name']>'']
fol_tmp.head(small).apply(lambda row:folium.Circle(location=[row["POI_LAT"], row["POI_LNG"]], 
                                              radius=10, fill_color='orange', color='orange',fill_opacity=1)
                                             .add_to(mapit) if row["POI_LAT"] else 0, axis=1)

fol_tmp.head(small).apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], 
                                              radius=1600, fill_color='blue', fill_opacity=0.025)
                                             .add_to(mapit), axis=1)
fol_tmp.head(small).apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], 
                                              radius=10, fill_color='red', color='red', fill_opacity=1)
                                             .add_to(mapit), axis=1)

18        <folium.vector_layers.Circle object at 0x7f344...
27        <folium.vector_layers.Circle object at 0x7f344...
43092     <folium.vector_layers.Circle object at 0x7f344...
43092     <folium.vector_layers.Circle object at 0x7f344...
43098     <folium.vector_layers.Circle object at 0x7f344...
                                ...                        
129152    <folium.vector_layers.Circle object at 0x7f344...
129152    <folium.vector_layers.Circle object at 0x7f344...
129152    <folium.vector_layers.Circle object at 0x7f344...
129152    <folium.vector_layers.Circle object at 0x7f344...
129152    <folium.vector_layers.Circle object at 0x7f344...
Length: 250, dtype: object

In [38]:
mapit

In [39]:
HI_f.columns

Index(['index_left', 'POI_LNG', 'POI_LAT', 'POI_name', 'listing_id',
       'Occupancy', 'Fall_weekday', 'Fall_weekend', 'Spring_weekday',
       'Spring_weekend', 'Summer_weekday', 'Summer_weekend', 'Winter_weekday',
       'Winter_weekend', 'Fall', 'Spring', 'Summer', 'Winter', 'Weekday',
       'Weekend', 'NoVacancy', 'id', 'listing_url', 'scrape_id',
       'last_scraped', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
      

In [41]:
HI_f.groupby(['listing_id','neighbourhood_group','room_type','Occupancy','NoVacancy'])\
        .agg({'POI_name':pd.Series.nunique}).reset_index()\
        .sort_values(by=['POI_name'], ascending=False)

,listing_id,neighbourhood_group,room_type,Occupancy,NoVacancy,POI_name
15326,42880322,Honolulu,Entire home/apt,203.0,0.556164,28
4360,16689497,Honolulu,Entire home/apt,205.0,0.561644,28
14367,40903353,Honolulu,Entire home/apt,120.0,0.328767,28
9199,28546176,Honolulu,Entire home/apt,171.0,0.468493,28
4038,15858101,Honolulu,Private room,83.0,0.227397,28
...,...,...,...,...,...,...
9960,30507295,Maui,Entire home/apt,181.0,0.495890,0
9962,30507299,Maui,Entire home/apt,136.0,0.372603,0
9963,30507305,Maui,Entire home/apt,119.0,0.326027,0
9969,30510256,Hawaii,Entire home/apt,142.0,0.389041,0


In [42]:
HI_f.to_csv("./Hawaii_nkm_v1.csv.gz", compression='gzip')